In [1]:
# Basic imports
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

/tmp/ipykernel_1662/3049972813.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Data Evaluation function
def data_evaluation(data):
    """Function to See the Data Evaluation, sees shape, head and describe

    Args:
        data (dataframe): Pandas Dataframe
    """
    print(f"{data.Name} Shape: {data.shape}")
    print(f"{data.Name} Head:\n{data.head()} ")
    print(f"{data.Name} Summary Statistics:\n{data.describe()}")
    print("\n")
    print(f"{data.Name} Columns:\n{data.columns}")

In [3]:
%store -r humidity
%store -r temperature
humidity.Name = 'humidity'
temperature.Name = 'temperature'
# Data Processing starts from here
if 'context_humidity' in humidity.columns:
    # Drop context Frame
    humidity.drop('context_humidity', axis=1, inplace=True)
if 'context_temperature' in temperature.columns:
    temperature.drop('context_temperature', axis = 1, inplace=True)

# See the data after droping the context coloumn
print("After Dropping Context: ")
data_evaluation(humidity)
data_evaluation(temperature)

# Remove Zero from the dataset
humidity = humidity[humidity['humidity'] != 0]
temperature = temperature[temperature['temperature'] != 0]

humidity.Name = 'humidity'
temperature.Name = 'temperature'
# See the data after removing zeroes
print("Removing Zeroes: ")
data_evaluation(humidity)
data_evaluation(temperature)

TypeError: 'PickleShareDB' object is not subscriptable

In [4]:
#Normalization of data
# Convert Unix timestamps to datetime objects
humidity['Datetime'] = pd.to_datetime(humidity['timestamp'], unit='ms')
temperature['Datetime'] = pd.to_datetime(temperature['timestamp'], unit='ms')

# Sort both DataFrames by the 'Datetime' column
humidity.sort_values(by='Datetime', inplace=True)
temperature.sort_values(by='Datetime', inplace=True)

# Set the tolerance in seconds
tolerance = pd.Timedelta(seconds=10)

# Merge dataframes based on the common timestamps within the tolerance
merged_df = pd.merge_asof(humidity, temperature,
                          on='Datetime', direction='nearest', tolerance=tolerance)

# Drop the 'Datetime' column as it's no longer needed
merged_df.drop(columns=['Datetime'], inplace=True)
merged_df.Name = 'merged frame'
data_evaluation(merged_df)

X = merged_df[['humidity', 'temperature']]

X = X.dropna()

# Data Visualization before Scaling
scatter_matrix(merged_df.iloc[:,[1,4]])
plt.savefig("../output/n_plot_merged.png")
plt.show()

# Do the Scaling
# instantiate the scaler
scale = StandardScaler()
# compute the mean and std to be used later for scaling
scale.fit(X)
# This are raw values
print("Mean and std before scale ")
print(scale.mean_)
print(scale.scale_)
print("\n")

# Now we Transform/Scale the data around 0,0
X_scaled = scale.transform(X)
# Let’s do a sanity check to see if each feature is centered at 0 and has a std of 1:
print(X_scaled.mean(axis=0))  # Mean comes down almost to zero, as data is scaled around 0,0
print(X_scaled.std(axis=0))

NameError: name 'humidity' is not defined

In [ ]:
%store X_scaled

Stored 'X_scaled' (ndarray)
